In [7]:
pip install -r requirements.txt

  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached groq-0.9.0-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pandas-2.0.2.tar.gz (5.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached starlette-0.41.3-py3-none-any.whl.metadata (6.0 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pyda

In [1]:
import pandas as pd
df = pd.read_csv('synthetic_logs.csv')
df.head()

,timestamp,source,log_message,target_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status


In [11]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [12]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

-finding similiear pattern in logs for regrex classification

In [19]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

#loading pre trained model sentenceTransform model

model = SentenceTransformer('all-MiniLM-L6-v2')

# generating embedding for the logs files
embedding = model.encode(df['log_message'].to_list())

In [21]:
embedding[:2].shape

(2, 384)

In [22]:
# performing DBscan for clustering
dbscan = DBSCAN(eps = 0.2, min_samples=1, metric = 'cosine')
clusters = dbscan.fit_predict(embedding)

df['cluster'] = clusters
df.head()

,timestamp,source,log_message,target_label,cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,0


In [24]:
# sorting clusters by numbers of record
cluster_counts = df['cluster'].value_counts()
large_clusters = cluster_counts[cluster_counts > 10].index

for cluster in large_clusters:
    print(f"Cluster {cluster}")
    print(df[df['cluster'] == cluster]['log_message'].head(5).to_string(index=False))
    print()

Cluster 0
nova.osapi_compute.wsgi.server [req-b9718cd8-f6...
nova.osapi_compute.wsgi.server [req-4895c258-b2...
nova.osapi_compute.wsgi.server [req-ee8bc8ba-92...
nova.osapi_compute.wsgi.server [req-f0bffbc3-5a...
nova.osapi_compute.wsgi.server [req-2bf7cfee-a2...

Cluster 5
nova.compute.claims [req-a07ac654-8e81-416d-bfb...
nova.compute.claims [req-d6986b54-3735-4a42-907...
nova.compute.claims [req-72b4858f-049e-49e1-b31...
nova.compute.claims [req-5c8f52bd-8e3c-41f0-95a...
nova.compute.claims [req-d38f479d-9bb9-4276-968...

Cluster 11
User User685 logged out.
 User User395 logged in.
 User User225 logged in.
User User494 logged out.
 User User900 logged in.

Cluster 13
Backup started at 2025-05-14 07:06:55.
Backup started at 2025-02-15 20:00:19.
  Backup ended at 2025-08-08 13:06:23.
Backup started at 2025-11-14 08:27:43.
Backup started at 2025-12-09 10:19:11.

Cluster 7
Multiple bad login attempts detected on user 85...
Multiple login failures occurred on user 9052 a...
  User 7153 

In [ ]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return label
    return None

In [34]:
classify_with_regex("User User123 logged in.")

'User Action'

In [41]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex.shape

(1910, 6)

Classification Stage 2: Classification Using Embeddings

In [44]:
print(df_non_regex['target_label'].value_counts()[df_non_regex['target_label'].value_counts() <= 5].index.tolist())

['Workflow Error', 'Deprecation Warning']


In [45]:
df_non_legacy = df_non_regex[df_non_regex.source!='LegacyCRM']
df_non_legacy.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI'], dtype=object)

In [46]:
# generating embedding for log message of legacy
filtered_embeddings = model.encode(df_non_legacy['log_message'].tolist())
filtered_embeddings[:2]

array([[-1.02939621e-01,  3.35459337e-02, -2.20260620e-02,
         1.55100937e-03, -9.86917689e-03, -1.78956255e-01,
        -6.34409934e-02, -6.01761453e-02,  2.81109326e-02,
         5.99620119e-02, -1.72618497e-02,  1.43365713e-03,
        -1.49560049e-01,  3.15288478e-03, -5.66031076e-02,
         2.71685179e-02, -1.49890939e-02, -3.54037322e-02,
        -3.62936482e-02, -1.45410597e-02, -5.61493309e-03,
         8.75538811e-02,  4.55120914e-02,  2.50963643e-02,
         1.00187482e-02,  1.24266921e-02, -1.39923543e-01,
         7.68696740e-02,  3.14095356e-02, -4.15246794e-03,
         4.36902568e-02,  1.71250310e-02, -8.00950676e-02,
         5.74006066e-02,  1.89091954e-02,  8.55262280e-02,
         3.96398939e-02, -1.34371847e-01, -1.44367816e-03,
         3.06704128e-03,  1.76854029e-01,  4.44888743e-03,
        -1.69274900e-02,  2.24266108e-02, -4.35049646e-02,
         6.09029876e-03, -9.98167880e-03, -6.23973496e-02,
         1.07372627e-02, -6.04893966e-03, -7.14661181e-0

In [48]:
# now trainging logistic regression then finding classification report
X = filtered_embeddings
y = df_non_legacy['target_label']

from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test,y_pred)
print(report)

                precision    recall  f1-score   support

Critical Error       0.91      1.00      0.95        48
         Error       0.98      0.89      0.93        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      0.99      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


synthetic_logs.csv


In [2]:
# now saving model 
import joblib
joblib.dump(clf, 'models/log_classifier.joblib')

NameError: name 'clf' is not defined

/workspaces/Log-Classification-System/training


/home/codespace/.local/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
